In [23]:
%pip install gurobipy

Note: you may need to restart the kernel to use updated packages.


In [24]:
import gurobipy as gp
from gurobipy import GRB

Considera una empresa de consultoría que tiene tres vacantes: Tester, Desarrollador y Arquitecto. Los tres mejores candidatos (recursos) para estos puestos son: Carlos, Jose y Monica. La empresa de consultoría realizó unas pruebas a cada candidato para evaluar su capacidad de desarrolar cada uno de los trabajos. 

Se supone que solo un candidato puede ser asignado a un trabajo y que, como máximo, un trabajo puede ser asignado a un candidato.

El problema consiste en determinar una asignación de recursos y trabajos de manera que cada trabajo sea cubierto, cada recurso sea asignado a lo sumo a un trabajo, y la suma total de las puntuaciones de coincidencia de las asignaciones sea maximizada.


![scores](matching_score_data.PNG)

In [25]:
# Resource and job sets
Candidatos = ['Carlos', 'Jose', 'Monica']
Trabajos = ['Tester', 'Desarrollador', 'Arquitecto']

In [26]:
# Matching score data
combinations, scores = gp.multidict({
    ('Carlos', 'Tester'): 53,
    ('Carlos', 'Desarrollador'): 27,
    ('Carlos', 'Arquitecto'): 13,
    ('Jose', 'Tester'): 80,
    ('Jose', 'Desarrollador'): 47,
    ('Jose', 'Arquitecto'): 67,
    ('Monica', 'Tester'): 53,
    ('Monica', 'Desarrollador'): 73,
    ('Monica', 'Arquitecto'): 47
})

In [ ]:
# Declara model
m = gp.Model('RAP')

In [ ]:
# Se crean las variables
x = m.addVars(combinations, name="assign")

In [ ]:
# Una única asignación al trabajo
m.addConstrs((x.sum('*',j) == 1 for j in Trabajos), name='asignacion_trabajos')

{'Tester': <gurobi.Constr *Awaiting Model Update*>,
 'Desarrollador': <gurobi.Constr *Awaiting Model Update*>,
 'Arquitecto': <gurobi.Constr *Awaiting Model Update*>}

In [30]:
# Restricciones de que para cada candidatos se le asigna un único trabajo (como mucho)
m.addConstrs((x.sum(r,'*') <= 1 for r in Candidatos), name='como_mucho_un_trabajo')


{'Carlos': <gurobi.Constr *Awaiting Model Update*>,
 'Jose': <gurobi.Constr *Awaiting Model Update*>,
 'Monica': <gurobi.Constr *Awaiting Model Update*>}

In [31]:
# Objetivo a maximizar
m.setObjective(x.prod(scores), GRB.MAXIMIZE)


In [32]:
# Save model for inspection
m.write('RAP.lp')


In [33]:
# Run optimization engine
m.optimize()



Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xb343b6eb
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6000000e+32   1.800000e+31   4.600000e+02      0s
       5    1.9300000e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.930000000e+02


In [34]:
# Display optimal values of decision variables
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal total matching score
print('Puntuación Matching total: ', m.objVal)

assign[Carlos,Tester] 1.0
assign[Jose,Arquitecto] 1.0
assign[Monica,Desarrollador] 1.0
Puntuación Matching total:  193.0
